In [35]:
#Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd
import requests
from pprint import pprint
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [36]:
# URL of page to be scraped; need to add each state to end of URL in loop
base_url = 'https://en.wikipedia.org/wiki/List_of_radio_stations_in_'

In [37]:
# CSV file of states for loop
states_df = pd.read_csv("List_of_states.csv")
states_df.head()

,States_w_space,State_w_no_space
0,Alabama,Alabama
1,Alaska,Alaska
2,Arizona,Arizona
3,Arkansas,Arkansas
4,California,California


In [38]:
# Initiate radio_df
url = base_url + states_df.State_w_no_space[0]
# read url
table = pd.read_html(url)
# Create a Dateframe From Table
radio_temp_df = pd.DataFrame(table[0])

#drop rows
radio_temp_df = radio_temp_df.drop(radio_temp_df.index[:])

# Loop through states
for index, row in states_df.iterrows():
    #Combine base url and state
    url = base_url + states_df.State_w_no_space[index]
    # read url
    table = pd.read_html(url)
    # Create a Dateframe From Table
    table_df = pd.DataFrame(table[0])
    # Add State
    table_df[5] = states_df.State_w_no_space[index]
    # append to radio_df
    radio_temp_df = radio_temp_df.append(table_df)
    
    

In [51]:
# Save dataframe to final
radio_df = radio_temp_df.copy()

In [52]:
#Rename columns and display table
radio_df = radio_df.rename(columns={radio_df.columns[0]:"Call_sign",radio_df.columns[1]:"Frequency", radio_df.columns[2]:"Location",radio_df.columns[3]:"License",radio_df.columns[4]:"Format",radio_df.columns[5]:"State"})
# Delete rows with Call sign
radio_df = radio_df[radio_df.Call_sign !='Call sign']
radio_df

,Call_sign,Frequency,Location,License,Format,State
1,KRLE,89.7 FM,Carbon Hill,Educational Media Foundation,Contemporary Christian (K-LOVE),Alabama
2,WAAO-FM,93.7 FM,Andalusia,"Three Notch Communications, LLC",Country,Alabama
3,WAAX,570 AM,Gadsden,Capstar TX LLC,News/Talk,Alabama
4,WABD,97.5 FM,Mobile,Cumulus Licensing LLC,Top 40 (CHR),Alabama
5,WABF,1480 AM,Mobile,Eternity Record Company LLC,Standards,Alabama
6,WACQ,580 AM,Tuskegee,"Tiger Communications, Inc.",Classic Hits,Alabama
7,WACT,1420 AM,Tuscaloosa,Capstar TX LLC,Adult Contemporary,Alabama
8,WACV,93.1 FM,Coosada,"Liberty Acquisitions 825, LLC",News/Talk,Alabama
9,WAFN-FM,92.7 FM,Arab,"Fun Media Group, Inc.",Oldies,Alabama
10,WAGF-FM,101.3 FM,Dothan,"Wilson Broadcasting Co., Inc.","R&B, Motown oldies, blues, gospel music, and s...",Alabama


In [53]:
# add revised format
#import radio csv as a dataframe
formats_df = pd.read_csv('formats.csv',encoding='latin_1')

#join radio dataframe on "Format"
radio_df = radio_df.merge(formats_df, on="Format", how="left")

#drop "Format"
radio_df = radio_df.drop(['Format'], axis=1)

#change Revised Format to Format
radio_df = radio_df.rename(columns={"Revised_Format":"Format"})

radio_df


,Call_sign,Frequency,Location,License,State,Format,Target_format
0,KRLE,89.7 FM,Carbon Hill,Educational Media Foundation,Alabama,Religious,No
1,WAAO-FM,93.7 FM,Andalusia,"Three Notch Communications, LLC",Alabama,Country,Yes
2,WAAX,570 AM,Gadsden,Capstar TX LLC,Alabama,Talk,No
3,WABD,97.5 FM,Mobile,Cumulus Licensing LLC,Alabama,Top40,No
4,WABF,1480 AM,Mobile,Eternity Record Company LLC,Alabama,Other,No
5,WACQ,580 AM,Tuskegee,"Tiger Communications, Inc.",Alabama,NaN,NaN
6,WACT,1420 AM,Tuscaloosa,Capstar TX LLC,Alabama,NaN,NaN
7,WACV,93.1 FM,Coosada,"Liberty Acquisitions 825, LLC",Alabama,Talk,No
8,WAFN-FM,92.7 FM,Arab,"Fun Media Group, Inc.",Alabama,Oldies,No
9,WAGF-FM,101.3 FM,Dothan,"Wilson Broadcasting Co., Inc.",Alabama,Variety,No


In [54]:
radio_df.Format.fillna(value="Other", inplace=True)
radio_df.Target_format.fillna(value="No", inplace=True)
radio_df


,Call_sign,Frequency,Location,License,State,Format,Target_format
0,KRLE,89.7 FM,Carbon Hill,Educational Media Foundation,Alabama,Religious,No
1,WAAO-FM,93.7 FM,Andalusia,"Three Notch Communications, LLC",Alabama,Country,Yes
2,WAAX,570 AM,Gadsden,Capstar TX LLC,Alabama,Talk,No
3,WABD,97.5 FM,Mobile,Cumulus Licensing LLC,Alabama,Top40,No
4,WABF,1480 AM,Mobile,Eternity Record Company LLC,Alabama,Other,No
5,WACQ,580 AM,Tuskegee,"Tiger Communications, Inc.",Alabama,Other,No
6,WACT,1420 AM,Tuscaloosa,Capstar TX LLC,Alabama,Other,No
7,WACV,93.1 FM,Coosada,"Liberty Acquisitions 825, LLC",Alabama,Talk,No
8,WAFN-FM,92.7 FM,Arab,"Fun Media Group, Inc.",Alabama,Oldies,No
9,WAGF-FM,101.3 FM,Dothan,"Wilson Broadcasting Co., Inc.",Alabama,Variety,No


In [50]:
radio_df_for_json = radio_df[radio_df.columns[4:]].to_json(orient='records')
radio_df_for_json

'[{"State":"Alabama","Format":"Religious"},{"State":"Alabama","Format":"Country"},{"State":"Alabama","Format":"Talk"},{"State":"Alabama","Format":"Top40"},{"State":"Alabama","Format":"Other"},{"State":"Alabama","Format":"Other"},{"State":"Alabama","Format":"Other"},{"State":"Alabama","Format":"Talk"},{"State":"Alabama","Format":"Oldies"},{"State":"Alabama","Format":"Variety"},{"State":"Alabama","Format":"Religious"},{"State":"Alabama","Format":"Other"},{"State":"Alabama","Format":"Other"},{"State":"Alabama","Format":"Variety"},{"State":"Alabama","Format":"Jazz"},{"State":"Alabama","Format":"Religious"},{"State":"Alabama","Format":"Urban"},{"State":"Alabama","Format":"Religious"},{"State":"Alabama","Format":"Other"},{"State":"Alabama","Format":"Other"},{"State":"Alabama","Format":"Top40"},{"State":"Alabama","Format":"Religious"},{"State":"Alabama","Format":"Other"},{"State":"Alabama","Format":"Talk"},{"State":"Alabama","Format":"Talk"},{"State":"Alabama","Format":"Talk"},{"State":"Alaba

In [ ]:
radio_df_for_json = radio_df[radio_df.columns[4:]].to_json(orient='records')
radio_df_for_json